In [ ]:
# Load a time series dataset and preprocess it for LSTM input.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load a sample time series dataset
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv"
data = pd.read_csv(url, header=0, parse_dates=["Month"], index_col="Month")

# Plot the dataset
data.plot(figsize=(10, 6))
plt.title("Monthly Airline Passengers")
plt.xlabel("Date")
plt.ylabel("Passengers")
plt.show()

# Convert data to a NumPy array
series = data["Passengers"].values

# Normalize the series
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range=(0, 1))
series_scaled = scaler.fit_transform(series.reshape(-1, 1))

In [ ]:
# Transform the time series data into input-output pairs suitable for LSTM training.

def create_sequences(data, time_steps=10):
    X, y = [], []
    for i in range(len(data) - time_steps):
        X.append(data[i:i + time_steps])
        y.append(data[i + time_steps])
    return np.array(X), np.array(y)

time_steps = 10
X, y = create_sequences(series_scaled, time_steps)

# Split into training and testing sets
split_ratio = 0.8
split_idx = int(len(X) * split_ratio)

X_train, X_test = X[:split_idx], X[split_idx:]
y_train, y_test = y[:split_idx], y[split_idx:]

print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)

In [ ]:
# Define an LSTM architecture for forecasting time series data.

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Build the LSTM model
model = Sequential([
    LSTM(50, activation='relu', return_sequences=True, input_shape=(time_steps, 1)),
    LSTM(50, activation='relu'),
    Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
model.summary()


In [ ]:
# Train the LSTM model on the training dataset.

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=2)


In [ ]:
# Evaluate the model’s performance on the test dataset using metrics like RMSE.

from sklearn.metrics import mean_squared_error

# Make predictions
y_pred = model.predict(X_test)

# Inverse transform the predictions and actual values
y_pred_rescaled = scaler.inverse_transform(y_pred)
y_test_rescaled = scaler.inverse_transform(y_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_rescaled, y_pred_rescaled))
print("Test RMSE:", rmse)


In [ ]:
# Plot the actual vs. predicted values to assess the model’s performance.

plt.figure(figsize=(10, 6))
plt.plot(y_test_rescaled, label="Actual")
plt.plot(y_pred_rescaled, label="Predicted")
plt.title("Actual vs Predicted - Time Series Forecasting")
plt.xlabel("Time Steps")
plt.ylabel("Passengers")
plt.legend()
plt.show()


In [ ]:
# Use the trained model to forecast future values.

def forecast_future(data, model, steps, time_steps=10):
    input_seq = data[-time_steps:]
    predictions = []
    for _ in range(steps):
        pred = model.predict(input_seq.reshape(1, time_steps, 1))[0]
        predictions.append(pred)
        input_seq = np.append(input_seq[1:], pred)
    return np.array(predictions)

# Forecast the next 12 months
future_steps = 12
future_preds = forecast_future(series_scaled, model, future_steps)
future_preds_rescaled = scaler.inverse_transform(future_preds)

# Plot future predictions
plt.figure(figsize=(10, 6))
plt.plot(data.index, series, label="Historical Data")
future_dates = pd.date_range(data.index[-1], periods=future_steps + 1, freq="M")[1:]
plt.plot(future_dates, future_preds_rescaled, label="Future Predictions")
plt.title("Time Series Forecasting - Future Predictions")
plt.xlabel("Date")
plt.ylabel("Passengers")
plt.legend()
plt.show()
